In [1]:
import os
# Find the latest version of spark 3.2.x  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.2'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 https://developer.downlo

In [2]:
# Install the PostgreSQL driver in our Colab environment
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-28 16:17:53--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.87MB/s    in 0.2s    

2022-11-28 16:17:53 (4.87 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Establish a Spark session and add the Postgres driver to the filepath
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [6]:
# Read the first CSV file from an S3 bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
mobile_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
mobile_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

In [7]:
# Count number of rows
print(mobile_df.count())

104975


In [9]:
# Drop incomplete rows and show updated count
drop_na = mobile_df.dropna()
print(drop_na.count())

104972


In [10]:
# Drop duplicates and show updated count
drop_duplicates = drop_na.dropDuplicates()
print(drop_duplicates.count())

104972


In [11]:
# Check schema
drop_duplicates.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [12]:
from pyspark.sql.functions import *

In [13]:
drop_duplicates = drop_duplicates.withColumn("review_date", to_date(col('review_date'), 'yyyy-mm-dd'))

In [14]:
drop_duplicates.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [15]:
# Transform dataset to fit schema
mobile_review_id_df = drop_duplicates.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
mobile_review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R5I2YBT9J85FF|   32397073|B011W26BU2|     938399424| 2015-01-31|
|R2W7SNCOCHAVOK|    2476375|B00XHAR9GI|     820413549| 2015-01-31|
|R1ED9U68DFV8V4|    2265767|B00J46XO9U|     744008282| 2015-01-28|
|R1JP6MNSIT0U2D|   45101321|B012C5XYHQ|     869734493| 2015-01-26|
|R34Z5GSD6QVJC0|   45941344|B00VU7PLZG|     973149575| 2015-01-26|
|R3FAV28GYZDFNS|   48325083|B00JJ1QBMW|     922547736| 2015-01-23|
|R1YTJAG0C7P2AZ|   23746886|0528007262|     513854777| 2015-01-17|
|R3CDVZ6UANW970|   19066740|B00QERR5CY|      82850235| 2015-01-14|
|R2S8VQ5ULUYZ78|   25870736|B002MYFAMA|     196145088| 2015-01-13|
|R1P1JSHR67KIZC|   24437477|B0126LLXGA|     490339933| 2015-01-13|
|R3UO8GRRT862EQ|   36297498|B00BHZT710|      27670198| 2015-01-11|
|R2I27L6J4U50D0|   29622286|B00XULD8GO|     399663747| 2015-01

In [16]:
mobile_products_df = drop_duplicates.select(['product_id', 'product_title'])
mobile_products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B011W26BU2|Bluetooth Speaker...|
|B00XHAR9GI|Towallmark 7" HD ...|
|B00J46XO9U|iXCC Lightning Ca...|
|B012C5XYHQ|Apple Ipod Nano 7...|
|B00VU7PLZG|[New Element Seri...|
|B00JJ1QBMW|DgRock® MRB600 Mi...|
|0528007262|Rand McNally TND ...|
|B00QERR5CY|iXCC Multi pack L...|
|B002MYFAMA|Buck Algonquin Da...|
|B0126LLXGA|Outdoor Bluetooth...|
|B00BHZT710|Tram Satellite Ra...|
|B00XULD8GO|Pyramid W64 6.5" ...|
|B002IUG14O|Garmin Nuvi Winds...|
|B00S0M9YK2|G.G.Martinsen 32 ...|
|B00B4E9764|BlackVue Wi-Fi DR...|
|B00JP7TCDK|EC TECHNOLOGY? Ne...|
|B00J46XO9U|iXCC Lightning Ca...|
|B00J46XO9U|iXCC Lightning Ca...|
|B001Q5DQOU|Sirius Radio Dot ...|
|B0063XRTKC|iPod touch 2/3 Ca...|
+----------+--------------------+
only showing top 20 rows



In [23]:
mobile_products_df = mobile_products_df.dropDuplicates(['product_id', 'product_title'])
mobile_products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B01075GRAO|ANYSECU QYT KT890...|
|B00QVXNV60|BRG AH2000G-ANR A...|
|B004ECO5VA|Holux Mini Smart ...|
|B00LGNKLXG|Aweek® Car Antenn...|
|B00M1Y87I0|      sentey ls 4063|
|B002ABX29I|Micca Slim Portab...|
|B00XXA2T2G|Centurion GT1 Car...|
|B00Q5A5PD6|HYUNDAI HCR-580 V...|
|B00VFEK8IY|Z ZTDM Mini Stere...|
|B00L85M8VU|Settonbrothers Ul...|
|B00SUXD5GU|[NEW RELEASE] JEB...|
|B00KJ7MVYW|Fincibo (TM) Appl...|
|B00A6YQQS0|Minisuit Aero Fol...|
|B009S99IYI|PIO TSG1644R 6.5 ...|
|B006PAZCXY|eForCity® Anti-Gl...|
|B00OXD61OE|SHE'O® 2x Cool No...|
|B000IBMI1K|RCA 200-Watt Mono...|
|B00UN6955M|Aisster(TM) Lot 1...|
|B00V684NCQ|LSD 8GB Flash Dri...|
|B00A609WOO|eForCity Hybrid S...|
+----------+--------------------+
only showing top 20 rows



In [19]:
mobile_customers = drop_duplicates.groupBy('customer_id').count()
mobile_customers_df = mobile_customers.withColumnRenamed('count', 'customer_count')
mobile_customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   11990227|             1|
|   12704032|             1|
|   23671289|             1|
|   14745975|             1|
|    3179193|             1|
|    2516709|             1|
|   16959619|             1|
|   31166502|             1|
|   33149353|             1|
|   45838973|             1|
|   10313569|             1|
|   49419305|             1|
|   51588676|             2|
|   44296544|             1|
|   11954795|             1|
|     538703|             1|
|   22485819|             1|
|   38079859|             1|
|   50328739|             1|
|   19396045|             1|
+-----------+--------------+
only showing top 20 rows



In [20]:
mobile_vine_df = drop_duplicates.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'])
mobile_vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R5I2YBT9J85FF|          5|            2|          2|   N|
|R2W7SNCOCHAVOK|          5|            0|          1|   N|
|R1ED9U68DFV8V4|          1|            0|          0|   N|
|R1JP6MNSIT0U2D|          1|            5|          6|   N|
|R34Z5GSD6QVJC0|          5|            0|          0|   N|
|R3FAV28GYZDFNS|          1|            0|          0|   N|
|R1YTJAG0C7P2AZ|          5|            0|          0|   N|
|R3CDVZ6UANW970|          5|            0|          0|   N|
|R2S8VQ5ULUYZ78|          5|            0|          0|   N|
|R1P1JSHR67KIZC|          5|            0|          0|   N|
|R3UO8GRRT862EQ|          5|            1|          1|   N|
|R2I27L6J4U50D0|          5|            0|          0|   N|
|R33W6ODHU78OOY|          2|            0|          0|   N|
| R4XW7GMHJASHT|          3|            

In [21]:
# Define AWS RDS parameters - Parameters deleted for submission - Please fill in appropriate values before running
my_aws_endpoint = 
my_aws_port_number = 
my_aws_database_name = 
my_aws_username = 
my_aws_password = 

In [22]:
# Define the connection string
jdbc_url=f'jdbc:postgresql://{my_aws_endpoint}:{my_aws_port_number}/{my_aws_database_name}'

# Set up the configuration parameters
config = {"user": f'{my_aws_username}', 
          "password": f'{my_aws_password}', 
          "driver":"org.postgresql.Driver"}

mode = 'append'

In [ ]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS
mobile_review_id_df.write.jdbc(url = jdbc_url, table = 'review_id_table', mode = mode, properties = config)

In [ ]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS
mobile_products_df.write.jdbc(url = jdbc_url, table = 'products', mode = mode, properties = config)

In [ ]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS
mobile_customers_df.write.jdbc(url = jdbc_url, table = 'customers', mode = mode, properties = config)

In [ ]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS
mobile_vine_df.write.jdbc(url = jdbc_url, table = 'vine_table', mode = mode, properties = config)